In [ ]:
# hello!! awaz?
# del test_dict['Mani']

# Tv = {'BreakingBad':100, 'GameOfThrones':1292, 'TMKUC' : 88}
 
# Keymax = max(zip(Tv.values(), Tv.keys()))[1]
# print(Keymax)


#TOP 5 TWEETS
# top_five = []
# def max_five(count_dict):
#     for i in range (5):
#         Keymax = max(zip(count_dict.values(), count_dict.keys()))[1]
#         top_five.append(Keymax)
#         del count_dict[Keymax]


#Remove punctuation and digits
# import re

# def remove_punctuation_and_numbers(input_string):
#     # Regular expression pattern to match punctuation and digits
#     pattern = r'[^\w\s]+|\d+'

#     # Remove the matched pattern from the input string
#     cleaned_string = re.sub(pattern, '', input_string)

#     return cleaned_string

# # Test the function
# input_string = "Hello, world! 123"
# output_string = remove_punctuation_and_numbers(input_string)
# print(output_string)  # Output: Hello world


In [17]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import numpy as np

In [12]:
import nltk
# nltk.download('stopwords') #run once 
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

from emot.emo_unicode import UNICODE_EMOJI

[nltk_data] Downloading package wordnet to /home/yash/nltk_data...


In [3]:
import re

In [ ]:
hashtags = []
attags = []

count_hash = {}
count_at = {}

def count_tags(tweet_text):
    hash_tag = re.findall(r"#(\w+)", tweet_text)
    at_tag = re.findall(r"@(\w+)", tweet_text)
    
    if(len(hash_tag)!=0):
        for i in hash_tag: 
            hashtags.append(i)
        for i in at_tag: 
            attags.append(i)

def freq(hashtags, attags):
    for i in range(0, len(hashtags)):
        if hashtags[i] not in count_hash.keys():
            count_hash[hashtags[i]] = 1
        else :
            count_hash[hashtags[i]] += 1
    

    for i in range(0, len(attags)):
        if attags[i] not in count_at.keys():
            count_at[attags[i]] = 1
        else :
            count_at[attags[i]] += 1
        
    # print(count_dict)

        

        

In [ ]:
query = "adani"

tweets = []

for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):   
    if i>10000:
        break

    else:
        # extract specific tweets
        #retweet ko lena?
        tweets.append([tweet.date, tweet.id, tweet.url, tweet.user.username, tweet.sourceLabel, tweet.user.location, tweet.content, tweet.likeCount, tweet.retweetCount])
        count_tags(tweet.content)

df = pd.DataFrame(tweets, columns = ['Date', 'ID', 'url', 'username', 'source', 'location', 'tweet', 'num_of_likes', 'num_of_retweet'])
df.to_csv('sentiment.csv', mode = 'a')

In [ ]:
# print(hashtags)
# print(attags)

freq(hashtags, attags)

# print(attags)
# print()
# print(hashtags)

top_hash = []
top_at = []
def max_five(count_tag, top_list):
    for i in range (20):
        Keymax = max(zip(count_tag.values(), count_tag.keys()))[1]
        # print(zip(count_tag.values(), count_tag.keys()))
        top_list.append((Keymax, count_tag[Keymax]))
        del count_tag[Keymax]

max_five(count_hash, top_hash)
max_five(count_at, top_at)

print()
print(top_hash)
print()
print(top_at)

In [7]:
eng_stop_words = list(stopwords.words('english'))
emoji = list(UNICODE_EMOJI.keys())

In [20]:
# regex
# def number_of_hashtags():
#     hashtags = []
    
#     tweets[‘tweet_text’].apply(lambda x: re.findall(r"#(\w+)", x))


def filtering(text):
    text = text.lower()

    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split())
    
    # print(text)
    
    # remove stop words
    tokens = word_tokenize(text)  
    filtered_words = [w for w in tokens if w not in eng_stop_words]
    filtered_words = [w for w in filtered_words if w not in emoji]

    lemmatizer = WordNetLemmatizer() 
    lemma_words = [lemmatizer.lemmatize(w) for w in filtered_words]
    text = " ".join(lemma_words)

    # print()
    # print(text)
    
    return text
    
    #polarity



filtering("@kvenkat1680 @SecularFringe @GemsOfBollywood Those will loss Who were faking despite of political alignments. As per Hindenberg, Adani manipulated &; used a fake assets scene and https://www.sdhvb.com he is getting rewards for his deeds, no surprise. As per your views the super hit movies are unable to stay on screen for more than two weeks. #Truth")


'loss faking despite political alignment per hindenberg adani manipulated used fake asset scene getting reward deed surprise per view super hit movie unable stay screen two week'

In [21]:
df = pd.read_csv('sentiment.csv')
df['Processed_Tweets'] = df['tweet'].apply(filtering)

In [23]:
df.head(10)

,Unnamed: 0,Date,ID,url,username,source,location,tweet,num_of_likes,num_of_retweet,Processed_Tweets
0,0.0,2023-02-03 08:02:50+00:00,1621418925283241986,https://twitter.com/kuldipp92846909/status/162...,kuldipp92846909,Twitter for Android,NaN,@warispathan Adani घोटाला किया तो तुम चुप क्यो...,0,0,adani
1,1.0,2023-02-03 08:02:49+00:00,1621418919402803201,https://twitter.com/HARSHAD_UBHADIA/status/162...,HARSHAD_UBHADIA,Twitter for Android,NaN,@jayvasavada Bap bap(ambani) hota hai beta bet...,0,0,bap bap ambani hota hai beta beta adani
2,2.0,2023-02-03 08:02:48+00:00,1621418916596838401,https://twitter.com/business/status/1621418916...,business,Twitter Web App,New York and the World,The fallout has also led to the removal of Ada...,1,0,fallout also led removal adani enterprise dow ...
3,3.0,2023-02-03 08:02:47+00:00,1621418912855519232,https://twitter.com/Dharmen62644900/status/162...,Dharmen62644900,Twitter for Android,NaN,@AdaniOnline @gautam_adani अच्छा।शेयर के भाव इ...,0,0,adani
4,4.0,2023-02-03 08:02:47+00:00,1621418912704528385,https://twitter.com/JerryMaguire911/status/162...,JerryMaguire911,Twitter for iPhone,Imaginative,@PurnimaNath @gautam_adani @rraina1481 ji. Is ...,0,0,adani ji possible
5,5.0,2023-02-03 08:02:47+00:00,1621418911949537282,https://twitter.com/anusan123/status/162141891...,anusan123,Twitter for iPhone,"Mumbai, India",@MahuaMoitra Didi inko ye konsa naya kaam pakd...,0,0,didi inko ye konsa naya kaam pakda diya aapne ...
6,6.0,2023-02-03 08:02:47+00:00,1621418910015946752,https://twitter.com/_pallavighosh/status/16214...,_pallavighosh,Twitter for iPhone,NaN,Govt says - it’s got nothing to do with the ad...,0,0,govt say got nothing adani controversy opp rai...
7,7.0,2023-02-03 08:02:46+00:00,1621418904693219328,https://twitter.com/IamSidSahib/status/1621418...,IamSidSahib,Twitter for iPhone,NaN,"In the end, everyone is a tool for this “adhir...",0,0,end everyone tool adhiraj
8,8.0,2023-02-03 08:02:45+00:00,1621418904139743233,https://twitter.com/roflghostrider/status/1621...,roflghostrider,Twitter for Android,NaN,@SureshChavhanke Guruji aap 1 episode Adani ke...,0,0,guruji aap 1 episode adani ke karke dikha de b...
9,9.0,2023-02-03 08:02:45+00:00,1621418903120543745,https://twitter.com/Shrey_msr/status/162141890...,Shrey_msr,Twitter for Android,"Bhubaneswar, Odisha",Everybody is talking about #Adani and #BJP loo...,0,0,everybody talking looting country nobody thoug...


In [ ]:
# election oriented

# like search 2024 elections trump biden .....

# removing emotionless words

# freq of top words
# segregate according to state and freq of top words

# overall polarity
# segregate according to state and polarity

# election advertising